In [14]:
import pickle
from datasets.pipelines import PreNormalize3D, RandomRot, GenSkeFeat, UniformSampleDecode,\
    FormatGCNInput, Collect, ToTensor
# from datasets.pipelines.sampling import UniformSampleDecode
# from utils.visualize import Vis3DPose

In [15]:
# annotations = './data/nturgbd/ntu60_3danno.pkl'
annotations = './data/pku/pkuv1_3danno.pkl'
with open(annotations, "rb") as f:
    data = pickle.load(f)

In [12]:
data['split'].keys()

dict_keys(['xsub_train', 'xsub_val', 'xset_train', 'xset_val', 'xview_train', 'xview_val'])

In [13]:
len(data['split']['xsub_train']) + len(data['split']['xsub_val']) + len(data['split']['xview_train']) + len(data['split']['xview_val'])

113156

In [16]:
data['annotations'][0].keys()

dict_keys(['label', 'frame_dir', 'keypoint', 'total_frames'])

In [9]:
len(data['annotations'])

56578

In [16]:
frames = []
for x in data['annotations']:
    frames.append(x['total_frames'])
max(frames), min(frames), sum(frames) / len(frames)

(300, 1, 112.38943557370962)

In [18]:
cnt = 0
for frame in frames:
    if frame > 100:
        cnt += 1
cnt, len(frames) - cnt

(13106, 43472)

In [6]:
anno = data['annotations'][10]
anno

{'frame_dir': 'S001C001P001R001A011',
 'label': 10,
 'keypoint': array([[[[2.0581e-01, 1.9202e-01, 3.8105e+00],
          [2.1338e-01, 4.4238e-01, 3.7266e+00],
          [2.2131e-01, 6.8750e-01, 3.6328e+00],
          ...,
          [6.8115e-02, 3.9038e-01, 3.4863e+00],
          [5.2338e-02, 3.3496e-01, 3.5117e+00],
          [1.1133e-01, 3.4131e-01, 3.5625e+00]],
 
         [[2.0581e-01, 1.9214e-01, 3.8105e+00],
          [2.1387e-01, 4.4263e-01, 3.7266e+00],
          [2.2229e-01, 6.8848e-01, 3.6328e+00],
          ...,
          [8.9172e-02, 3.9673e-01, 3.5195e+00],
          [5.1392e-02, 3.1885e-01, 3.5332e+00],
          [1.0736e-01, 3.3423e-01, 3.5723e+00]],
 
         [[2.0605e-01, 1.9238e-01, 3.8105e+00],
          [2.1399e-01, 4.4287e-01, 3.7266e+00],
          [2.2229e-01, 6.8848e-01, 3.6328e+00],
          ...,
          [1.0004e-01, 3.8989e-01, 3.5254e+00],
          [5.2521e-02, 3.2690e-01, 3.5293e+00],
          [1.0706e-01, 3.3472e-01, 3.5723e+00]],
 
         ...,
 
  

In [3]:
anno = data['annotations'][0]
anno = PreNormalize3D()(anno)
anno.keys()

dict_keys(['frame_dir', 'label', 'keypoint', 'total_frames', 'body_center'])

In [4]:
anno = RandomRot(theta=0.2)(anno)
anno.keys()

dict_keys(['frame_dir', 'label', 'keypoint', 'total_frames', 'body_center'])

In [5]:
anno = GenSkeFeat()(anno)
anno.keys()

dict_keys(['frame_dir', 'label', 'total_frames', 'body_center', 'keypoint'])

In [6]:
anno['keypoint'].shape

(1, 103, 25, 3)

In [7]:
anno = UniformSampleDecode(clip_len=100)(anno)
anno.keys(), anno['keypoint'].shape

(dict_keys(['frame_dir', 'label', 'total_frames', 'body_center', 'clip_len', 'frame_interval', 'num_clips', 'keypoint']),
 (1, 100, 25, 3))

In [8]:
anno = FormatGCNInput()(anno)
anno.keys(), anno['keypoint'].shape

(dict_keys(['frame_dir', 'label', 'total_frames', 'body_center', 'clip_len', 'frame_interval', 'num_clips', 'keypoint']),
 (1, 2, 100, 25, 3))

In [9]:
anno = Collect(keys=['keypoint', 'label'], meta_keys=[])(anno)
anno.keys()

dict_keys(['keypoint', 'label'])

In [17]:
anno = ToTensor(keys=['keypoint', 'label'])(anno)

In [18]:
anno['label']

tensor([0])

In [1]:
import torch

In [3]:
x = torch.randn(2, 750, 256)
N, L, D = x.shape

In [4]:
T, V = 30, 25
mask_ratio = 0.8
V_len_keep = int(V * (1 - mask_ratio))
V_len_keep

4

In [14]:
t_segment = 6
T_ = T // t_segment
noise = torch.rand(N, T_, V)

In [15]:
ids_shuffle = torch.argsort(noise, dim=-1)
ids_restore = torch.argsort(ids_shuffle, dim=-1)

In [16]:
ids_shuffle.shape, ids_restore.shape

(torch.Size([2, 5, 25]), torch.Size([2, 5, 25]))

In [18]:
ids_shuffle = ids_shuffle.repeat_interleave(t_segment, dim=1)
ids_restore = ids_restore.repeat_interleave(t_segment, dim=1)
ids_shuffle.shape, ids_restore.shape

(torch.Size([2, 30, 25]), torch.Size([2, 30, 25]))

In [19]:
ids_keep = ids_shuffle[:, :, :V_len_keep]
ids_keep.shape

torch.Size([2, 30, 4])

In [20]:
x = x.view(N, T, V, D)
x.shape

torch.Size([2, 30, 25, 256])

In [21]:
x_mask = torch.gather(x, dim=2, index=ids_keep.unsqueeze(-1).expand(-1, -1, -1, D))
x_mask.shape

torch.Size([2, 30, 4, 256])

In [22]:
mask = torch.ones(N, T, V)
mask[:, :, :V_len_keep] = 0
mask = torch.gather(mask, dim=2, index=ids_restore)
mask.shape

torch.Size([2, 30, 25])

In [27]:
mask[:, :7, :5]

tensor([[[0., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1.]],

        [[0., 0., 1., 1., 0.],
         [0., 0., 1., 1., 0.],
         [0., 0., 1., 1., 0.],
         [0., 0., 1., 1., 0.],
         [0., 0., 1., 1., 0.],
         [0., 0., 1., 1., 0.],
         [1., 0., 1., 1., 1.]]])